In [2]:
import psycopg2
import pandas as pd

In [20]:
def create_database():
    #connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=postgres")
    conn.set_session(autocommit = True)
    cur = conn.cursor()

    #create new database
    cur.execute("DROP DATABASE IF EXISTS fortune500")
    cur.execute("CREATE DATABASE fortune500")

    #close default connection
    conn.close()

    #connect to new database
    conn = psycopg2.connect("host=127.0.0.1 dbname=fortune500 user=postgres password=postgres")
    cur = conn.cursor()

    return conn, cur

In [6]:
#read csv file
f500 = pd.read_csv(r'C:\Users\martin.bergmann\python_scripts\projects\postgres_data_ingestion\data\fortune500.csv')

In [ ]:
f500.head()

In [ ]:
f500.dtypes

In [7]:
#convert string values to numeric
f500['Revenue (in millions)'] = pd.to_numeric(f500['Revenue (in millions)'], errors='coerce')

In [ ]:
f500['Profit (in millions)'] = pd.to_numeric(f500['Profit (in millions)'], errors='coerce')
print(f500.dtypes)

In [ ]:
f500.describe()

In [9]:
#calculate proper values
f500['Revenue'] = f500['Revenue (in millions)'] * 1000000

In [10]:
f500['Profit'] = f500['Profit (in millions)'] * 1000000

In [11]:
#drop all unnecessary columns
f500.drop(['Revenue (in millions)', 'Profit (in millions)'], axis=1, inplace=True)

In [35]:
#create new database with function
conn, cur = create_database()
conn.commit()

In [15]:
#create table in database
f500_table = ("""CREATE TABLE IF NOT EXISTS f500(
    Year INT,
    Rank INT,
    Company VARCHAR,
    Revenue NUMERIC,
    Profit NUMERIC
    )""")

cur.execute(f500_table)
conn.commit()

In [14]:
#create insert statement
f500_table_insert = ("""INSERT INTO f500(
    Year,
    Rank,
    Company,
    Revenue,
    Profit)
    VALUES(%s, %s, %s, %s, %s)
    """)

In [16]:
#insert data into table
for index, row in f500.iterrows():
    cur.execute(f500_table_insert, list(row))



In [17]:
conn.commit()

In [37]:
#check if data got ingested
cur.execute("""SELECT * FROM f500 LIMIT 100""")


In [ ]:
results=cur.fetchall()
results


In [39]:
cur.close()
conn.close()